In [1]:
# Try to schedule multiple DDFs 

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp
from scipy.stats import binned_statistic
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
# from MAF. Should eventually go to sims_utils
def calcSeason(ra, time):
    """Calculate the 'season' in the survey for a series of ra/dec/time values of an observation.
    Based only on the RA of the point on the sky, it calculates the 'season' based on when this
    point would be overhead .. the season is considered +/- 0.5 years around this time.

    Parameters
    ----------
    ra : float
        The RA (in degrees) of the point on the sky
    time : np.ndarray
        The times of the observations, in MJD

    Returns
    -------
    np.ndarray
        The season values
    """
    # Reference RA and equinox to anchor ra/season reference - RA = 0 is overhead at autumnal equinox
    # autumn equinox 2014 happened on september 23 --> equinox MJD
    Equinox = 2456923.5 - 2400000.5
    # convert ra into 'days'
    dayRA = ra / 360 * 365.25
    firstSeasonBegan = Equinox + dayRA - 0.5 * 365.25
    seasons = (time - firstSeasonBegan) / 365.25
    # Set first season to 0
    seasons = seasons - np.floor(np.min(seasons))
    return seasons

In [4]:
ddf_data = np.load('ddf_grid.npz')
ddf_grid = ddf_data['ddf_grid'].copy()

# XXX-- double check that I got this right
ack = ddf_grid['sun_alt'][0:-1] * ddf_grid['sun_alt'][1:]
night = np.zeros(ddf_grid.size, dtype=int)
night[np.where((ddf_grid['sun_alt'][1:] >=0) & (ack < 0))] += 1
night = np.cumsum(night)

In [7]:
DDFs = ['DD:ELAISS1', 'DD:XMM-LSS']

array([(59853.5       ,  0.35789797, 0.        ,         nan,         nan,  0.        , nan, nan,  0.        ,  0., inf,  0.        ,  0., inf,  0.        ,  0., inf),
       (59853.51041667,  0.41360456, 0.        ,         nan,         nan,  0.        , nan, nan,  0.        ,  0., inf,  0.        ,  0., inf,  0.        ,  0., inf),
       (59853.52083333,  0.46894875, 0.        ,         nan,         nan,  0.        , nan, nan,  0.        ,  0., inf,  0.        ,  0., inf,  0.        ,  0., inf),
       ...,
       (63505.96874915, -0.12942338, 0.        ,  0.        ,         inf, -0.        , nan, nan, -0.        , nan, nan, -0.        , nan, nan, -0.        , nan, nan),
       (63505.97916582, -0.18569237, 0.        ,  0.        ,         inf, -0.        , nan, nan, -0.        , nan, nan, -0.        , nan, nan, -0.        , nan, nan),
       (63505.98958248, -0.24112162, 1.90646443, 20.87839022, 23.56106875, -9.35888477, nan, nan, -8.85712875, nan, nan, -1.36598529, nan, nan, -1.4

In [ ]:
schedule_list = []

# I can just make a giant schedule that has things stacked! 


for ddf_name in DDFs:
    RA = ddf_ra[ddf_name]  # RA of the DDF
    
    season = calcSeason(RA, ddf_grid['mjd'])
    season % 1 # Can set things to be 0 when out of season, and 1 in season. Then do a cumsum, normalize, maybe round.
    
    ngrid = ddf_grid['mjd'].size
    sun_limit = np.radians(-18.)
    airmass_limit = 2.1  
    sky_limit = 22. #20. #21.5 #22.
    zeropoint = 25.0  # mags
    sequence_limit = 400 #230
    pause_time = 13/24.  # days
    
    delta_t = ddf_grid['mjd'][1] - ddf_grid['mjd'][0]

    # Let's try scheduling just one for now
    schedule = m.addMVar(ngrid, vtype=GRB.BINARY, name="schedule_%s" % ddf_name)
    # set a sun mask
    sun_mask = np.zeros(ngrid, dtype=bool)
    sun_mask[np.where(ddf_grid['sun_alt'] >= sun_limit)] = 1

    airmass_mask = np.zeros(ngrid, dtype=bool)
    airmass_mask[np.where(ddf_grid['DD:ELAISS1_airmass'] >= airmass_limit)] = 1

    sky_mask = np.zeros(ngrid, dtype=bool)
    sky_mask[np.where(ddf_grid['DD:ELAISS1_sky_g'] <= sky_limit)] = 1
    sky_mask[np.where(np.isnan(ddf_grid['DD:ELAISS1_sky_g']) == True)] = 1

    # Let's add the constraints
    m.addConstr(schedule @ sun_mask == 0)
    m.addConstr(schedule @ airmass_mask == 0)
    m.addConstr(schedule @ sky_mask == 0)

    # limit the total number of ddf sequences
    # HA! Need to set an exact number I think. Or maybe a range.
    m.addConstr(schedule.sum() == sequence_limit)

    # prevent a repeat sequence in a night
    unights, indx = np.unique(night, return_index=True)
    night_mjd = ddf_grid['mjd'][indx]
    # The season of each night
    night_season = calcSeason(RA, night_mjd)
    sched_night = m.addMVar(unights.size, vtype=GRB.CONTINUOUS)
    for i,n in enumerate(unights):
        in_night = np.where(night == n)[0]
        m.addConstr(schedule[in_night]@schedule[in_night] <= 1)
        m.addConstr(sched_night[i] == schedule[in_night].sum())

# Declare a new master schedule that is the sum of all the individual ones, and put a constraint on it
# Or maybe just constrain all the different schedules to multiply to zero?